In [1]:
import SimpleITK as sitk
import os
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import ipyvolume as ipv
from scipy.ndimage import zoom
import gui
%matplotlib notebook

In [2]:
# Callback invoked by the IPython interact method for scrolling and modifying the alpha blending
# of an image stack of two images that occupy the same physical space. 
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
    plt.show()
    
# Callback invoked when the StartEvent happens, sets up our new data.
def start_plot():
    global metric_values, multires_iterations
    
    metric_values = []
    multires_iterations = []

# Callback invoked when the EndEvent happens, do cleanup of data and figure.
def end_plot():
    global metric_values, multires_iterations
    
    del metric_values
    del multires_iterations
    # Close figure, we don't want to get a duplicate of the plot latter on.
    plt.close()

# Callback invoked when the IterationEvent happens, update our data and display new figure.    
def plot_values(registration_method):
    global metric_values, multires_iterations
    
    metric_values.append(registration_method.GetMetricValue())                                       
    # Clear the output area (wait=True, to reduce flickering), and plot current data
    clear_output(wait=True)
    # Plot the similarity metric values
    plt.plot(metric_values, 'r')
    plt.plot(multires_iterations, [metric_values[index] for index in multires_iterations], 'b*')
    plt.xlabel('Iteration Number',fontsize=12)
    plt.ylabel('Metric Value',fontsize=12)
    plt.show()
    
# Callback invoked when the sitkMultiResolutionIterationEvent happens, update the index into the 
# metric_values list. 
def update_multires_iterations():
    global metric_values, multires_iterations
    multires_iterations.append(len(metric_values))
    
def myshow(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayViewFromImage(img)
    spacing = img.GetSpacing()
        
    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]
        
        # the the number of components is 3 or 4 consider it an RGB image
        if not c in (3,4):
            nda = nda[nda.shape[0]//2,:,:]
    
    elif nda.ndim == 4:
        c = nda.shape[-1]
        
        if not c in (3,4):
            raise Runtime("Unable to show 3D-vector Image")
            
        # take a z-slice
        nda = nda[nda.shape[0]//2,:,:,:]
            
    ysize = nda.shape[0]
    xsize = nda.shape[1]
      
    # Make a figure big enough to accommodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi

    fig = plt.figure(figsize=figsize, dpi=dpi)
    # Make the axis the right size...
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    
    extent = (0, xsize*spacing[1], ysize*spacing[0], 0)
    
    t = ax.imshow(nda,extent=extent,interpolation=None)
    
    if nda.ndim == 2:
        t.set_cmap("gray")
    
    if(title):
        plt.title(title)

In [3]:
series_reader = sitk.ImageSeriesReader()
cur_paths = series_reader.GetGDCMSeriesFileNames("../hubmap_data/lung_data/right_lung/R175")
img = sitk.ReadImage(cur_paths)
img = sitk.RescaleIntensity(img)
lung_175 = sitk.GetArrayViewFromImage(img)

In [4]:
series_reader = sitk.ImageSeriesReader()
cur_paths = series_reader.GetGDCMSeriesFileNames("../hubmap_data/lung_data/right_lung/R205")
img = sitk.ReadImage(cur_paths)
img = sitk.RescaleIntensity(img)
lung_205 = sitk.GetArrayViewFromImage(img)

In [5]:
import skimage

lung_205 = skimage.transform.downscale_local_mean(lung_205, factors(2,2,2))
lung_175 = skimage.transform.downscale_local_mean(lung_175, factors(2,2,2))

query = np.array([22, 17, 12]) * 10
neighbour = np.array([17, 21, 13]) * 10

/Users/eyal/anaconda3/envs/spatial/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


NameError: name 'factors' is not defined

In [ ]:
neighbour

array([340, 420, 260])

In [ ]:
plt.figure(figsize=(10, 5))
# regularization parameter

plt.subplot(1, 2, 1)
plt.imshow(lung_175[:,:,query[2]])
plt.scatter(query[0], query[1], s=10, c='red', marker='o')

plt.subplot(1, 2, 2)
plt.imshow(lung_205[:,:,neighbour[2]])
plt.scatter(neighbour[0], neighbour[1], s=10, c='red', marker='o')

plt.show()